In [3]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

#### 1. Load model

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI()

embeddings = OpenAIEmbeddings()



#### 2. Create retriever

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [6]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [7]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

#### 3. Create retriever_tool

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [13]:
retriever_tool

Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fbc67ceb310>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fbc67ceb310>)>)

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [11]:
tools = [retriever_tool, search]

In [14]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [15]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [17]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the exec

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.\n\n4. Test Prioritization: Lan

In [18]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/january-1/', 'content': 'San Francisco Weather in January  San Francisco weather in January San Francisco weather by month // weather averages 9.6 (49.2) 6.2 (43.2) 14 (57.3) 113  San Francisco weather in January // weather averages Airport close to San Francisco  you can find all information about the weather in San Francisco in January:Data: 1991 - 2021 Min. Temperature °C (°F), Max. Temperature °C (°F), Precipitation / Rainfall mm (in), Humidity, Rainy days. Data: 1999 - 2019: avg. Sun hours San Francisco weather and climate for further months San Francisco in February San Francisco in March San Francisco in April San Francisco in May San Francisco in June San Francisco in July'}]The weather in San Francisco varies throughout the year. In January, the 

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco varies throughout the year. In January, the average minimum temperature is around 9.6°C (49.2°F) and the average maximum temperature is around 14°C (57.3°F). The city experiences an average of 113 mm (4.4 inches) of precipitation in January. If you would like more detailed information about the weather in San Francisco for other months, please let me know.'}

In [19]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to help you make it more compelling and impactful. It can help you identify areas that need improvement, such as clarity, structure, and content.

2. Resume/CV Review: LangSmith can review your resume or CV and provide suggestions on how to make it more effective and tailored to the LLM application. It can help you highlight your relevant experiences, skills, and achievements.

3. Letter of Recommendation Guidance: LangSmith can provide guidance on selecting the right recommenders and help you draft a request for a letter of recommendation. It can also provide tips on what information to include in the request and how to follow up with your recommenders.

4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting the righ

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to help you make it more compelling and impactful. It can help you identify areas that need improvement, such as clarity, structure, and content.\n\n2. Resume/CV Review: LangSmith can review your resume or CV and provide suggestions on how to make it more effective and tailored to the LLM application. It can help you highlight your relevant experiences, skills, and achievements.\n\n3. Letter of Recommendation Guidance: LangSmith can provide guidance on selecting the right recommenders and help you draft a request for a letter of recommendation. It can also provide tips on what information to include in the request and how to follow up with your recommenders.\n\